In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

import logging
import os
import time



In [3]:
df = pd.read_csv('IMDB.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.shape

(50000, 2)

### **Data Preprocessing**

In [5]:
# define text preprocessing func

def lemmatization(text):
    "Lemmatize the text"
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)


def remove_stop_words(text):
    "remove stopwords from the text "
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)


def removing_numbers(text):
    "Removing numbers from the text"
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    "convert text to lower case"
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)


def removing_punctuations(text):
    "removing punctuations from the text"
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace(':',"")
    text = re.sub('\s+', ' ', text).strip()
    return text


def removing_urls(text):
    "Remove url from the text"
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df = normalize_text(df)
df.head()

,review,sentiment
0,one reviewer mentioned watching oz episode hoo...,positive
1,wonderful little production br br the filming ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there s family little boy jake think...,negative
4,petter mattei s love time money visually stunn...,positive


In [7]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})
df.head()

,review,sentiment
0,one reviewer mentioned watching oz episode hoo...,1
1,wonderful little production br br the filming ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically there s family little boy jake think...,0
4,petter mattei s love time money visually stunn...,1


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [11]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [13]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/ak0614653/Capstone-Project.mlflow')

dagshub.init(repo_owner='ak0614653', repo_name='Capstone-Project', mlflow=True)

mlflow.set_experiment('Logistic Regression Baseline')



Accessing as ak0614653

Initialized MLflow to track repo "ak0614653/Capstone-Project"

Repository ak0614653/Capstone-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/f94e7d3d52a04a6f80c14a2b40d83de4', creation_time=1741006397873, experiment_id='0', last_update_time=1741006397873, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [14]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


# configure logging
logging.info("Starting MLflow run...")

with mlflow.start_run():
	start_time = time.time()
	
	try:
		logging.info('Logging preprocessing parameters...')
		mlflow.log_param('vectorizor', 'Bag of words')
		mlflow.log_param("num_features", 100)
		mlflow.log_param("test_size", 0.3)
		
		logging.info("Initializing Logistic Regression model....")
		model = LogisticRegression(max_iter=1000) # Increase max_iter to prevent non-convergence issues
		
		logging.info("Fitting the model")
		model.fit(X_train, y_train)        
		logging.info("Model Training Complete")

		logging.info("Logging model parameters...")
		mlflow.log_param("model", "Logistic Regression")

		logging.info("Making Predictions")
		y_pred = model.predict(X_test)

		logging.info("Calculating evaluation Metrics...")
		accuracy = accuracy_score(y_test,y_pred)
		precision = precision_score(y_test,y_pred)
		recall = recall_score(y_test,y_pred)
		score = f1_score(y_test,y_pred)

		logging.info("Logging evaluation metrics...")

		mlflow.log_metric("accuracy",accuracy)
		mlflow.log_metric("precision",precision)
		mlflow.log_metric("recall",recall)
		mlflow.log_metric("f1_score",score)

		logging.info("Saving and logging the model...")
		mlflow.sklearn.log_model(model,"model")

		end_time = time.time()
		logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

		# Print the results for verification
		logging.info(f"Accuracy: {accuracy}")
		logging.info(f"Precision: {precision}")
		logging.info(f"Recall: {recall}")
		logging.info(f"F1 Score: {score}")

	except Exception as e:
		logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-09 22:58:57,280 - INFO - Starting MLflow run...
2025-03-09 22:58:58,321 - INFO - Logging preprocessing parameters...
2025-03-09 22:58:59,577 - INFO - Initializing Logistic Regression model....
2025-03-09 22:58:59,577 - INFO - Fitting the model
2025-03-09 22:58:59,941 - INFO - Model Training Complete
2025-03-09 22:58:59,942 - INFO - Logging model parameters...
2025-03-09 22:59:00,401 - INFO - Making Predictions
2025-03-09 22:59:00,406 - INFO - Calculating evaluation Metrics...
2025-03-09 22:59:00,459 - INFO - Logging evaluation metrics...
2025-03-09 22:59:02,239 - INFO - Saving and logging the model...
2025/03/09 22:59:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-09 22:59:26,522 - INFO - Model training and logging completed in 28.20 seconds.
2025-03-09 22:59:26,527 - INFO - Accuracy: 0.7350666666666666
2025-03-09 22:59:26,530 - INFO - Prec

🏃 View run beautiful-crane-338 at: https://dagshub.com/ak0614653/Capstone-Project.mlflow/#/experiments/0/runs/ab0a7ea5776b4531b8ec5f07de86adf3
🧪 View experiment at: https://dagshub.com/ak0614653/Capstone-Project.mlflow/#/experiments/0
